In [1]:
import os
import tweepy
from dotenv import load_dotenv

In [2]:
load_dotenv()

consumer_key = os.environ["API_KEY"]
consumer_secret = os.environ["API_KEY_SECRET"]
access_token = os.environ["ACCESS_TOKEN"]
access_token_secret = os.environ["ACCESS_TOKEN_SECRET"]
bearer_token = os.environ["BEARER_TOKEN"]


client = tweepy.Client(
  bearer_token=bearer_token,
  consumer_key=consumer_key,
  consumer_secret=consumer_secret,
  access_token=access_token,
  access_token_secret=access_token_secret,
  wait_on_rate_limit=True
)

In [23]:
def extraer_cuentas_usuarios(lista_cuentas, lista_tweets, response):
    for i in range(len(response)):
        #lista_cuentas.append(response.includes['users'][i].username)
        lista_cuentas.append(response.includes['users'][i])
        lista_tweets.append(response.data[i].text)

    print(len(lista_cuentas))

In [24]:
# Search Recent Tweets
# This endpoint/method returns Tweets from the last seven days
query = 'depresión yo tengo'
#query = 'Petro'
# The method returns a Response object, a named tuple with data, includes,
# errors, and meta fields

cuentas_usuarios = []
tweets = []
for response in tweepy.Paginator(client.search_recent_tweets,
                                query,
                                tweet_fields=['id', 'author_id', 'text'],
                                user_fields=['id', 'username', 'location'],
                                expansions='author_id',
                                max_results=100, limit=1):#.flatten(limit=100):
    extraer_cuentas_usuarios(cuentas_usuarios, tweets, response)



4


In [25]:
print(cuentas_usuarios)

[<User id=87266642 name=Miguel A López-López username=MiguelAngelopez>, <User id=2804417358 name=𝙂𝙤́𝙢𝙚𝙯 username=pepedesgracias>, <User id=2179618707 name=fiorella.estefania username=fiorella0017>, <User id=931309107377852417 name=Ernesto quicano ramirez username=QuicanoRamirez>]


In [26]:
print(tweets)

['Me emputa muchísimo, porque aunque yo no tenga depresión, sí tengo varias personas cercanas a mí que la tienen y me consta la lucha diaria que es para ser “funcional” en esta sociedad, para que igual cuando lo sean les cierren la puerta porque asumen que no van a poder.', 'El trabajo más pesado lo tengo yo, luchar contra la depresión y tener todos los días turno en una clínica de 7am a 9pm \U0001fae1', '-La Aitana: Vos sos la más adulta del grupo, sos monotributista y tenés trabajo estable.\n-Yo: Pero soy alcohólica y tengo depresión.\n-La Aitana: Bueno, ahí si tenemos un problema.', '@juancar27172435 @ElRepublicano09 Para Julián Román si yo tengo 10 vacas soy un Ben catway (bonanza),como yo creer qu el es harley Davison(el sobrevivió a la depresión económica de 1900)']


In [27]:
for usuario in cuentas_usuarios:
    print(usuario.location)

He / Him - Bogotá, Colombia
None
Tucumán - Argentina
None


In [28]:
usuarios = []
for usuario in cuentas_usuarios:
    if (usuario.location is not None):
        if ('Medellin' in usuario.location) or ('Medellín' in usuario.location) or ('medellin' in usuario.location) or ('medellín' in usuario.location):
            usuarios.append(usuario.username)

In [29]:
print(usuarios)

[]


In [31]:
usuarios_globales = set(usuarios)

In [32]:
print(usuarios_globales)

set()


In [ ]:


# #print(tweets)

# # Each Tweet object has default ID and text fields
# for tweet in tweets:
#     print(tweet.id)
#     print(tweet)
#     print(tweet.text)